In [1]:
#First step we define the spark session to establish the connection
# Fo this we need to import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.types import StructType

spark = SparkSession.builder.appName('stockcount').getOrCreate()


In [2]:
#import other libraries to be used
from textblob import TextBlob # Used to process text and do sentiment analysis
from googletrans import Translator #used to translate texts there is not in English
from unidecode import unidecode #used to non-usual 

In [3]:
#Create the dataframe responsable to read each line receied throght the local host and port 9995

twitters = spark.readStream.format("socket").option("host", "localhost").option("port", 4407).load()

In [4]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType, FloatType

In [5]:
#Translate function 
def translate_udf(col):
    trans_obj=Translator().translate(col)
    return trans_obj.text

#function to sentiment analysis
def sentiment_udf(col):
    sentiment_text=TextBlob(col)
    return sentiment_text.polarity

In [6]:
#definition of the function as user-defined-function
unicode_udf_string = udf(lambda z: unidecode(z), StringType()) #Defines the function of decode to be used in dataframe
group_by_sentiment = udf(lambda x: 'bearish' if x < -0.1 else 'Bullish' if x > 0.1 else 'Neutral',StringType())
translate_udf_string = udf(translate_udf, StringType())
sentiment_udf_float = udf(sentiment_udf, FloatType()) #Translate function


In [6]:
test = 'O mercado esta indo abaixo'
decode = unidecode(test)
decodeEN=Translator().translate(decode)
print(decodeEN.text)
x=str(decodeEN)
sentiment = TextBlob(x)
print(sentiment.polarity)

The market is going down
-0.15555555555555559


In [7]:
test2 = 'Nous nous attendons à une accumulation pour briser la zone de résistance devient un support'
decode = unidecode(test2)
decodeEN=Translator().translate(decode)
print(decodeEN.text)
x=str(decodeEN)
sentiment = TextBlob(x)
print(sentiment.polarity)

We expect an accumulation to break the resistance zone becomes a support
0.0


In [33]:
test4 = 'Überraschend hoher DXY-Index, Vertrauen in Vermögenswerte '
decode = unidecode(test4)
decodeEN=Translator().translate(decode)
print(decodeEN.text)
x=str(decodeEN)
sentiment = TextBlob(x)
print(sentiment.polarity)

Surprisingly high DXY index, trust in assets
0.16


In [34]:
twitters_unicode = twitters.select("value", unicode_udf_string(twitters.value).alias("unicoded")) #decode
twitters_uni_trans=twitters_unicode.select("value", "unicoded", translate_udf_string(col("unicoded")).alias("stock_twits_EN")) #Translate
twitters_uni_trans_sent=twitters_uni_trans.select("value", "unicoded", "stock_twits_EN", sentiment_udf_float(col("stock_twits_EN")).alias("analysis")) #Analysis
t_sent_label=twitters_uni_trans_sent.select("value", "unicoded", "stock_twits_EN", "analysis", group_by_sentiment(col("analysis")).alias("classified")) #classifica

In [35]:
t_sent_count = t_sent_label.groupBy('classified').count()

In [ ]:
#Define schema for the data 
files_dir = spark.readStream\
    .format("txt")\
    .schema(userSchema)\
    .option('includeTimestamp', 'true')\
    .option("header", "true")\
    .option("delimiter", ";")\
    .option("maxFilesPerTrigger", 1)\
    .load("C:\Users\Admin\Desktop\DATA_ANALYST\IRON_HACK\Course_ware\Week_3\Project\Spark-StockTwitterC:\Users\Admin\Desktop\DATA_ANALYST\IRON_HACK\Course_ware\Week_3\Project\Spark-StockTwitter\*.csv")

In [ ]:
query = t_sent_count.writeStream.outputMode('update').format("console").option('truncate','false').start()

query.awaitTermination() #Waits until the streaming query ends